In [102]:
import pandas as pd
import numpy as np
import time

In [14]:
from bs4 import BeautifulSoup as soup
import requests

In [92]:
# create urls we need
base_url = "https://www.transfermarkt.co.uk/premier-league/einnahmenausgaben/wettbewerb/GB1/plus/0?ids=a&sa=&" 
url_end = "&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0/"

# tags of form "saison_id=*start_year*&saison_id_bis=*start_year*"
url_dict = dict()
for k in range(1995,2019):
    tag = "saison_id=" + str(k) + "&saison_id_bis=" + str(k)
    url_dict[k] = base_url + tag + url_end

In [63]:
# get the page html
response = requests.get(url_dict[2017], headers={'User-Agent': 'Custom'})
page = soup(response.content, "html.parser")

In [88]:
# get all table entries
odd_entries = page.findAll("tr",{"class":"odd"})
even_entries = page.findAll("tr",{"class":"even"})
entries = odd_entries + even_entries

In [82]:
# write code to get each of the data points we want
test_entry = odd_entries[0]
# get expenditure
expenditure_block = test_entry.findAll("td",{"class":"rechts hauptlink redtext"})
expenditure = expenditure_block[0].text
expenditure

'£285.75m'

In [83]:
# get income
income_block = test_entry.findAll("td",{"class":"rechts hauptlink greentext"})
income = income_block[0].text
income

'£86.72m'

In [85]:
# get team name
team_block = test_entry.findAll("td",{"class":"hauptlink no-border-links"})
team_name = team_block[0].a.text
team_name

'Manchester City'

In [93]:
# put this all together into one function
def get_tuple_data(entry):
    team_block = entry.findAll("td",{"class":"hauptlink no-border-links"})
    team_name = team_block[0].a.text
    expenditure_block = entry.findAll("td",{"class":"rechts hauptlink redtext"})
    expenditure = expenditure_block[0].text
    income_block = entry.findAll("td",{"class":"rechts hauptlink greentext"})
    income = income_block[0].text
    return (team_name,expenditure,income)

In [94]:
# now need a function to apply above function to each entry
def get_all_tuples(entries):
    tuples = []
    for entry in entries:
        data = get_tuple_data(entry)
        tuples.append(data)
    return tuples

In [95]:
# now need a function to, for a single page, get all the data on that page
def get_page_data(url):
    # get the page html
    response = requests.get(url, headers={'User-Agent': 'Custom'})
    page = soup(response.content, "html.parser")
    
    # get all table entries
    odd_entries = page.findAll("tr",{"class":"odd"})
    even_entries = page.findAll("tr",{"class":"even"})
    entries = odd_entries + even_entries
    
    tuples = get_all_tuples(entries)
    return tuples

In [100]:
# a function to get all page data, for all urls
def get_all_pages_data(url_dict):
    df = pd.DataFrame(data=[],columns=['Year','Team','Expenditure','Income'])
    for key in url_dict.keys():
        year_col = [str(key)]*20
        url = url_dict.get(key)
        tuples = get_page_data(url)
        data = [(year_col[i],tuples[i][0],tuples[i][1],tuples[i][2]) for i in range(20)]
        df_new = pd.DataFrame(data=data,columns=['Year','Team','Expenditure','Income'])
        df = pd.concat([df,df_new],axis=0)
        print("Year {} Complete".format(key))
        time.sleep(60)
    return df

In [103]:
# now using what we have, we can write all this data to a csv
csv_file = 'transfer_data.csv'
df = get_all_pages_data(url_dict)
df.to_csv(csv_file,index=False)

Year 1995 Complete
Year 1996 Complete
Year 1997 Complete
Year 1998 Complete
Year 1999 Complete
Year 2000 Complete
Year 2001 Complete
Year 2002 Complete
Year 2003 Complete
Year 2004 Complete
Year 2005 Complete
Year 2006 Complete
Year 2007 Complete
Year 2008 Complete
Year 2009 Complete
Year 2010 Complete
Year 2011 Complete
Year 2012 Complete
Year 2013 Complete
Year 2014 Complete
Year 2015 Complete
Year 2016 Complete
Year 2017 Complete
Year 2018 Complete
